In [10]:
%pip install scikit-learn


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import numpy as np
import sys
sys.path.append('../')
from utils import remove_similar_rows, find_lines_with_player


In [12]:
url = 'https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/data_files/all_data_v3.csv'
df = pd.read_csv(url)


In [13]:
# Filter out the German data and reindex
df_ger = df[df["language"] == "de"]


df_ger = df_ger.reset_index(drop=True)

In [14]:
# Remove the similiar rows (The Function is imported from utils on top)
df = remove_similar_rows(pd.DataFrame(df_ger['data']), df_ger)

data_lower = df.copy()

data_lower['data'] = data_lower['data'].str.lower()
data_lower['player'] = data_lower['player'].str.lower()
data_lower.head()

data_wo_pattern = data_lower.copy()

data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"^{\'content\': \'", "", str(x)))
data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r"\\nAdvertisement\\nAdvertisement\\nAdvertisement|nutze kicker'\}$", "", str(x)))


In [ ]:
#data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(r'^\s*nutze kicker.*?\.', "", str(x), flags=re.M))


In [ ]:
data_wo_pattern['data'].iloc[1]

'nutze kicker auf seinen digitalen plattformen wie gewohnt mit werbung und tracking. die zustimmung kann jederzeit für die zukunft widerrufen werden. details zu werbe- und analyse-trackern findest du in unserer datenschutzerklärung oder bei cookies & tracking am ende jeder seite.\\nmit dem pur-abo nutzt du kicker auf seinen digitalen plattformen ohne werbetracking und praktisch werbefrei.\\nnur €2,49 im monat\\nbereits pur-abonnent? hier anmelden.\\nalle antworten zum pur-abo findest du hier.\\nwerbung und tracking: um dir relevante inhalte und personalisierte werbung anzubieten, setzen wir cookies und andere technologien ein. damit messen wir, wie und womit du unsere angebote nutzt. diese daten geben wir auch an dritte weiter. wir greifen dabei auf dein endgerät zu, speichern cookies oder sonstige informationen und wir oder dritte können mit diesen sowie mit persönlichen identifikatoren (z.b. geräte-kennungen oder ip-adressen) und basierend auf deinem individuellen nutzungsverhalten .

## Find sentences with Playername

In [ ]:
data_with_playernames = find_lines_with_player(data_wo_pattern, data_wo_pattern['player'].unique())

In [ ]:
data_with_playernames['data'].iloc[0]

'glücksmoment! exequiel palacios hebt nach seinem tor zum 2:1 ab'

## Pattern Regex


In [ ]:
patternlist = [
    r'^\s*nutze kicker.*?\.',
    r'^\s*die zustimmung.*?\.',
    r'^\s*details zu werbe-.*?\.',
    r'^\s*\\nmit dem.*?\.',
    r'^\s*\\nnur €2,49.*?\.',
    r'^\s*\\nbereits pur-.*?\.',
    r'^\s*\\nalle antworten.*?\.',
    r'^\s*\\nwerbung.*?\.',
    r'^\s*damit messen.*?\.',
    r'^\s*diese daten .*?\.', 
    r'^\s*wir greifen .*?\.', 
    r'^\s*b..*?\.', 
    r'^\s*\\nhinweis.*?\.',
    r'^\s*..\\nhinweis.*?\.',
    r'^\s*die usa.*?\.',
    r'^\s*es besteht.*?\.',    
    # add more patterns here
]

In [ ]:
for pattern in patternlist:

    data_wo_pattern['data'] = data_wo_pattern['data'].apply(lambda x: re.sub(pattern, "", str(x), flags=re.M))


## Stopwords

In [ ]:
german_stop_words = stopwords.words('german')

# Create a list of stop words to remove 
stop_words_to_remove = ['nicht', 'nichts']

# Remove the stop words to remove from the stop words list
for word in stop_words_to_remove:
  german_stop_words.remove(word)

data_rm = df_ger.copy()

# Define a function to apply remove_stopwords on a column
def remove_stopwords_from_text(text):
    return remove_stopwords(text, stopwords=german_stop_words)

# Apply the remove_stopwords function to the 'text' column using the apply method
data_rm['data'] = data_rm['data'].apply(remove_stopwords_from_text)


In [ ]:
data_rm.head()

,data,player,language,publishedAt
0,{'content': 'Trainer Xabi Alonso mitgereisten ...,Exequiel Palacios,de,2023-02-24T09:33:31Z
1,{'content': 'Nutze kicker digitalen Plattforme...,Exequiel Palacios,de,2023-03-03T21:35:13Z
2,{'content': 'Nutze kicker digitalen Plattforme...,Exequiel Palacios,de,2023-03-07T11:34:39Z
3,{'content': 'Mitchel Bakker Mittwoch Abschluss...,Exequiel Palacios,de,2023-03-08T14:25:18Z
4,{'content': '© G+J Medien GmbH\nZugang STERN P...,Exequiel Palacios,de,2023-03-09T19:36:53Z
